In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
data=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data= np.array(data)

In [4]:
m,n=data.shape

In [5]:
np.random.shuffle(data)

In [6]:
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [7]:
Y_train

array([3, 0, 4, ..., 1, 0, 9])

In [8]:
def init_params():
    W1 = np.random.rand(10,784) - 0.5
    B1 = np.random.rand(10,1) - 0.5
    W2 = np.random.rand(10,10) - 0.5
    B2 = np.random.rand(10,1) - 0.5
    
    return W1,B1,W2,B2

In [9]:
def ReLU(Z):
    return np.maximum(Z,0) 

# denominator - sum for each column 

In [10]:
def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    

In [11]:
def forward_propgation(W1,B1,W2,B2,X):
    Z1 = W1.dot(X) + B1
    A1 = ReLU(Z1)
    
    Z2 = W2.dot(A1) + B2
    A2 = softmax(Z2)
    
    return Z1,A1,Z2,A2
    

# Y.size = m = number of training examples , Y.max + 1 to get 10 output classes
    
# np.arange(Y.size) = 0 to m (all rows basically) then access Yth column to set it to 1



In [12]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [13]:
def derivative_ReLU(Z):
    return Z > 0 

In [14]:
def backward_propgation(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    dB2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * derivative_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    dB1 = 1 / m * np.sum(dZ1)
    return dW1, dB1, dW2, dB2
    
    

In [15]:
def update_parameters(W1,B1,W2,B2,dW1,dB1,dW2,dB2, alpha):
    W1 = W1 - alpha * dW1
    B1 = B1 - alpha * dB1
    W2 = W2 - alpha * dW2
    B2 = B2 - alpha * dB2
    
    return W1,B1,W2,B2

In [16]:
def get_predictions(A2):
    return np.argmax(A2,0)

In [17]:
def get_accuracy(predictions,Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

In [18]:
def gradient_descent(X, Y, alpha, iterations):
    W1, B1, W2, B2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_propgation(W1, B1, W2, B2, X)
        dW1, dB1, dW2, dB2 = backward_propgation(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, B1, W2, B2 = update_parameters(W1, B1, W2, B2, dW1, dB1, dW2, dB2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, B1, W2, B2

In [19]:
W1,B1,W2,B2 = gradient_descent(X_train,Y_train,0.10, 500)

Iteration:  0
[0 0 7 ... 2 0 0] [3 0 4 ... 1 0 9]
0.08382926829268293
Iteration:  10
[1 0 7 ... 1 0 0] [3 0 4 ... 1 0 9]
0.24073170731707316
Iteration:  20
[1 0 7 ... 1 0 0] [3 0 4 ... 1 0 9]
0.30721951219512195
Iteration:  30
[1 0 2 ... 1 0 0] [3 0 4 ... 1 0 9]
0.3742439024390244
Iteration:  40
[1 0 2 ... 1 0 6] [3 0 4 ... 1 0 9]
0.4143414634146341
Iteration:  50
[3 0 2 ... 1 0 6] [3 0 4 ... 1 0 9]
0.4492439024390244
Iteration:  60
[3 0 2 ... 1 0 6] [3 0 4 ... 1 0 9]
0.4889268292682927
Iteration:  70
[3 0 2 ... 1 0 6] [3 0 4 ... 1 0 9]
0.5282926829268293
Iteration:  80
[3 0 2 ... 1 0 6] [3 0 4 ... 1 0 9]
0.5622682926829269
Iteration:  90
[3 0 4 ... 1 0 9] [3 0 4 ... 1 0 9]
0.5928292682926829
Iteration:  100
[3 0 4 ... 1 0 4] [3 0 4 ... 1 0 9]
0.6192439024390244
Iteration:  110
[3 0 4 ... 1 0 4] [3 0 4 ... 1 0 9]
0.6423658536585366
Iteration:  120
[3 0 4 ... 1 0 4] [3 0 4 ... 1 0 9]
0.660829268292683
Iteration:  130
[3 0 4 ... 1 0 4] [3 0 4 ... 1 0 9]
0.6762439024390244
Iteration:  140